# 15. clij on Colab

One of the great advantages of Colab is the availability of GPUs. While that possibility is mainly exploited to run Tensorflow or PyTorch based deep learning algorithms, other software can benefit from it.

[clij](https://clij.github.io/) is a library available on multiple platforms that implements a large collection of GPU-based image processing functions. Interestingly, while most GPU software is based on NVIDIA's CUDA library which limits its use to NVIDIA GPUs, clij is based on OpenCL, an open-source library for GPU computing. This makes GPU computing available to GPUs from virtually any manufacturer, and allows everyone to exploit that resource even on a laptop.

clij is mainly centered on Fiji but is also available for other software such as Icy or Matlab. And there is also an experimental version for Python named [clijpy](https://github.com/clij/clijpy). In this notebooks we demonstrate how to use clij in a notebook and in particular on Colab where one can benefit from powerful GPUs.

## Installing components

We have seen in the previous notebook how it was possible to run Fiji and Python together thanks to PyImageJ. On a regular computer, this could be achieved by simply installing that package as well as Java via conda. This installation is more complicated on Colab which offers a more constrained workspace, e.g. where it is not very easy to use conda. The main part of the installation solution comes from [this post](https://forum.image.sc/t/pyimagej-on-google-colab/32804/8?u=guiwitz) on the image.sc forum and [this notebook](https://colab.research.google.com/drive/1zo7DZRDK5FJTfEj5gUe4iLl7V4_7r48T). **Note that you only have to do this installation when you start a runtime from scratch. If you installed the components, you can "Runtime -> Restart runtime" and they will still be available. In that case jump to the next [section](#clij-via-macro).**

In [ ]:
# importing os to set environment variable
import os

# install openjdk for java and set ENV variable
!apt-get install -y openjdk-8-jdk-headless -qq > /dev/null      #install openjdk
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"     #set environment variable

# install maven
!sudo apt-get install maven

# install imglyb
!pip install imglyb==0.3.5

# install pyimagej
!pip install pyimagej

# install pyjnius (python library to access java classes)
!wget https://anaconda.org/conda-forge/pyjnius/1.2.0/download/linux-64/pyjnius-1.2.0-py36hc2cfd8d_0.tar.bz2 && tar -xjf pyjnius-1.2.0-py36hc2cfd8d_0.tar.bz2 -C /usr/local

In order to be able to use clij, we need to specifically install that plugin into Fiji. A solution to do this is to locally install Fiji and then install plugins from the command line. First we download Fiji and unzip it:

In [ ]:
!wget https://downloads.imagej.net/fiji/latest/fiji-linux64.zip && unzip fiji-linux64.zip

Then we need to add the plugins. For this we first add the clij update sites (the addresses that one finds when manually adding plugins in Fiji). Then we update all plugins to install them.

In [ ]:
%%bash
cd Fiji.app/
./ImageJ-linux64 --update add-update-site clij2 https://sites.imagej.net/clij2/
./ImageJ-linux64 --update add-update-site clij https://sites.imagej.net/clij/
./ImageJ-linux64 --update update

## clij via macro

Now that we have a local installation of Fiji and of PyImageJ, we can run the same kind of code as in the last notebook, i.e. we can create a Fiji-macro using clij and run it directly in the notebook. First we start Fiji and the WindowManager class. On Colab we also need to specify the location of Pyjnius to be able to access Java classes. **If you just performed the above installation, please "Runtime -> Restart Runtime" now.**

In [ ]:
import os
os.environ["PYJNIUS_JAR"] = "/usr/local/share/pyjnius/pyjnius.jar"

import imagej;
ij = imagej.init("Fiji.app")

from jnius import autoclass
WindowManager = autoclass('ij.WindowManager')

Again, we use a very simple macro where we import an image and then filter it with a Gaussian. Only this time, the filtering is done via clij:

In [ ]:
# define the macro as text
macro = """
#@ Float sigma
run("Blobs (25K)");
input = getTitle();
blurred = "Blurred";

run("CLIJ Macro Extensions", "cl_device=");
Ext.CLIJ_clear();

Ext.CLIJ_push(input);
run("Close All");

Ext.CLIJ_blur2D(input, blurred, sigma, sigma);
Ext.CLIJ_pull(blurred);

Ext.CLIJ_clear();


"""
args = {
    'sigma': 20.0,
}
ij.py.run_macro(macro, args);

# recover the current window containig the blurred image
blurred = WindowManager.getCurrentImage()

# plot it
ij.py.show(blurred)

## Using clijpy

Instead of using a Fiji macro, we can directly use the clijpy library which has been designed to work as a direct Python interface for clij. Note that this demonstration is heavily based on [a notebook](https://github.com/clij/clijpy/blob/master/python/clijpy_demo.ipynb) available in the clijpy repository. First, we import the clij Java classes, and then get a Clij instance (similarly to what is done in the macro:

In [ ]:
CLIJx = autoclass('net.haesleinhuepf.clijx.CLIJx')
clijx = CLIJx.getInstance();

Then we can write the entire pipeline in Python. We only have to be careful about conversions of images between the Numpy and Java worlds. Remember that clijpy exploits PyImageJ and therefore we deal with the same concepts here. Let's first import an image as a numpy array:

In [ ]:
from skimage import io
import numpy as np

np_arr = io.imread('https://samples.fiji.sc/blobs.png')

Then we can convert that image to Java, push it to the GPU, create a "target" image in which the blurred version of the image will be placed, and do the filtering:

In [ ]:
ij_img = ij.py.to_java(np_arr)

# push the image to the GPU
input8 = clijx.push(ij_img)

# create a float image
inputF = clijx.create(input8.getDimensions())

# copy image to float image
clijx.copy(input8, inputF)

# reserve memory for output, same size and type as input
blurred = clijx.create(inputF);

# filter the image on the GPU
clijx.blur(inputF, blurred, 5, 5, 0);

# pull image out of GPU
imageplus_result = clijx.pull(blurred);

# display the result
ij.py.show(imageplus_result)